In [2]:
!pip install -q anthropic
!pip install -q openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 4.4 MB/s eta 0:00:00


In [3]:
import anthropic
import openai
import json
import pandas as pd
from google.colab import userdata
from google import genai

In [4]:
api_key = userdata.get('ANTHROPIC_API_KEY')
client = anthropic.Anthropic(api_key=api_key)

openai_api_key = userdata.get('OPENAI_API_KEY')
openai_client = openai.OpenAI(api_key = openai_api_key)

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
google_client = genai.Client(api_key=GOOGLE_API_KEY)

In [5]:
def claude_prompt(question, follow_up):
    prompt = f"""You are an expert in natural language processing. Please label the following question-follow-up pair as "1" if the follow-up asks the same or a similar question to the initial question, or if it explicitly or implicitly states that the question was not answered. Otherwise, label "0". It is okay to label 0 if the follow-up is asking for futher clarification or specifics.

Initial question: "{question}"
Follow-up: "{follow_up}"

Provide your label (1 or 0):"""
    return prompt

def claude_label(question, follow_up):
    message = client.messages.create(
        model="claude-3-7-sonnet-20250219",
        max_tokens=150,
        temperature=0,
        system="You analyze question-follow-up pairs and determine if the follow-up is asking the same/similar question or indicating the initial question wasn't answered (label '1'), or if it's a new topic or asking for further details (label '0'). Respond with ONLY the label (1 or 0).",
        messages=[
            {"role": "user", "content": claude_prompt(question, follow_up)}
        ]
    )

    response = message.content[0].text
    return response


def openai_label(question, follow_up):
    prompt = f"Label the following question-follow-up pair as '1' if the follow-up repeats the question or suggests the question was not answered. Otherwise, label '0'.\n\n. It is okay to label 0 if the follow-up is asking for futher clarification or specifics, or if the follow-up is unrelated."
    prompt += f"Question: {question}\nFollow-up: {follow_up}\nLabel:"

    response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are an NLP model trained to label question-follow-up pairs."},
            {"role": "user", "content": prompt}
        ]
    )

    label = response.choices[0].message.content.strip()
    return label


def google_label(question, follow_up):
    prompt = f"Label the following question-follow-up pair as '1' if the follow-up repeats the question or suggests the question was not answered. Otherwise, label '0'.\n\n. It is okay to label 0 if the follow-up is asking for futher clarification or specifics, or if the follow-up is unrelated."
    prompt += f"Question: {question}\nFollow-up: {follow_up}\nLabel:"

    response = google_client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt,
      )

    return response.text[0]


In [7]:
with open('combined_interviews.json', 'r') as f:
    data = json.load(f)

df = pd.DataFrame(data)

In [8]:
df["label_claude"] = df.apply(lambda row: claude_label(row["question"], row["follow-up"]), axis=1)


In [9]:
df["label_openai"] = df.apply(lambda row: openai_label(row["question"], row["follow-up"]), axis=1)


In [10]:
df["label_google"] = df.apply(lambda row: google_label(row["question"], row["follow-up"]), axis=1)


In [11]:
df

,interviewer,question,interviewee,response,follow-up,label_claude,label_openai,label_google
0,Nicole Gelinas,"Thank you, comptroller. So you talked about th...",Brad Lander,"Thank you for the question, and thank you for ...",Which crimes have gone up the most since the p...,0,0,0
1,Josh Greenman,Which crimes have gone up the most since the p...,Brad Lander,"Yeah. I mean, obviously, there was a huge rise...",Do you know which violent crimes have seen the...,1,1,1
2,Josh Greenman,Do you know which violent crimes have seen the...,Brad Lander,"I mean, from pre-pandemic levels in general, t...",Felony assaults have gone up a lot. Shootings ...,0,0,0
3,Josh Greenman,Felony assaults have gone up a lot. Shootings ...,Brad Lander,I think it’s the same strategy. It’s like you’...,"If you're talking with Ken Corey, are you worr...",0,0,0
4,Myles Miller,"If you're talking with Ken Corey, are you worr...",Brad Lander,I think the problems with the Caban era of the...,Would you consider keeping Tisch as police com...,0,0,0
...,...,...,...,...,...,...,...,...
453,Christina Greer,That’s 54 countries.,Whitney Tilson,"Oh, you’re right. I’ve been to 86 countries, b...",Do you think you can be anti-Zionist and not a...,0,0,0
454,Nicole Gelinas,Do you think you can be anti-Zionist and not a...,Whitney Tilson,"I think it is possible. I think in practice, m...","But is a loud, raucous protest with posters th...",0,0,0
455,Josh Greenman,"But is a loud, raucous protest with posters th...",Whitney Tilson,“From the river to the sea.” “Globalize the In...,If you’re the mayor of New York City. What’s y...,0,0,0
456,Josh Greenman,If you’re the mayor of New York City. What’s y...,Whitney Tilson,"Yeah, that’s the key. If they’re peaceful, the...",Thank you. Thank you.,0,1,0


In [12]:
df[df['label_claude']==df["label_openai"]]

,interviewer,question,interviewee,response,follow-up,label_claude,label_openai,label_google
0,Nicole Gelinas,"Thank you, comptroller. So you talked about th...",Brad Lander,"Thank you for the question, and thank you for ...",Which crimes have gone up the most since the p...,0,0,0
1,Josh Greenman,Which crimes have gone up the most since the p...,Brad Lander,"Yeah. I mean, obviously, there was a huge rise...",Do you know which violent crimes have seen the...,1,1,1
2,Josh Greenman,Do you know which violent crimes have seen the...,Brad Lander,"I mean, from pre-pandemic levels in general, t...",Felony assaults have gone up a lot. Shootings ...,0,0,0
3,Josh Greenman,Felony assaults have gone up a lot. Shootings ...,Brad Lander,I think it’s the same strategy. It’s like you’...,"If you're talking with Ken Corey, are you worr...",0,0,0
4,Myles Miller,"If you're talking with Ken Corey, are you worr...",Brad Lander,I think the problems with the Caban era of the...,Would you consider keeping Tisch as police com...,0,0,0
...,...,...,...,...,...,...,...,...
451,Akash Mehta,You’re comfortable talking about Zohran Mamdan...,Whitney Tilson,"I guess on a policy basis, I’m reasonably clos...",You ranked Mayor Koch highly. Koch was one of ...,0,0,0
452,Harry Siegel,And this will be our last word to be fair to y...,Whitney Tilson,"I would say, in order, Ukraine, Israel, and Af...",That’s 54 countries.,0,0,0
453,Christina Greer,That’s 54 countries.,Whitney Tilson,"Oh, you’re right. I’ve been to 86 countries, b...",Do you think you can be anti-Zionist and not a...,0,0,0
454,Nicole Gelinas,Do you think you can be anti-Zionist and not a...,Whitney Tilson,"I think it is possible. I think in practice, m...","But is a loud, raucous protest with posters th...",0,0,0


In [13]:
df[df['label_claude']==df["label_google"]]

,interviewer,question,interviewee,response,follow-up,label_claude,label_openai,label_google
0,Nicole Gelinas,"Thank you, comptroller. So you talked about th...",Brad Lander,"Thank you for the question, and thank you for ...",Which crimes have gone up the most since the p...,0,0,0
1,Josh Greenman,Which crimes have gone up the most since the p...,Brad Lander,"Yeah. I mean, obviously, there was a huge rise...",Do you know which violent crimes have seen the...,1,1,1
2,Josh Greenman,Do you know which violent crimes have seen the...,Brad Lander,"I mean, from pre-pandemic levels in general, t...",Felony assaults have gone up a lot. Shootings ...,0,0,0
3,Josh Greenman,Felony assaults have gone up a lot. Shootings ...,Brad Lander,I think it’s the same strategy. It’s like you’...,"If you're talking with Ken Corey, are you worr...",0,0,0
4,Myles Miller,"If you're talking with Ken Corey, are you worr...",Brad Lander,I think the problems with the Caban era of the...,Would you consider keeping Tisch as police com...,0,0,0
...,...,...,...,...,...,...,...,...
453,Christina Greer,That’s 54 countries.,Whitney Tilson,"Oh, you’re right. I’ve been to 86 countries, b...",Do you think you can be anti-Zionist and not a...,0,0,0
454,Nicole Gelinas,Do you think you can be anti-Zionist and not a...,Whitney Tilson,"I think it is possible. I think in practice, m...","But is a loud, raucous protest with posters th...",0,0,0
455,Josh Greenman,"But is a loud, raucous protest with posters th...",Whitney Tilson,“From the river to the sea.” “Globalize the In...,If you’re the mayor of New York City. What’s y...,0,0,0
456,Josh Greenman,If you’re the mayor of New York City. What’s y...,Whitney Tilson,"Yeah, that’s the key. If they’re peaceful, the...",Thank you. Thank you.,0,1,0


In [14]:
df[df['label_openai']==df["label_google"]]

,interviewer,question,interviewee,response,follow-up,label_claude,label_openai,label_google
0,Nicole Gelinas,"Thank you, comptroller. So you talked about th...",Brad Lander,"Thank you for the question, and thank you for ...",Which crimes have gone up the most since the p...,0,0,0
1,Josh Greenman,Which crimes have gone up the most since the p...,Brad Lander,"Yeah. I mean, obviously, there was a huge rise...",Do you know which violent crimes have seen the...,1,1,1
2,Josh Greenman,Do you know which violent crimes have seen the...,Brad Lander,"I mean, from pre-pandemic levels in general, t...",Felony assaults have gone up a lot. Shootings ...,0,0,0
3,Josh Greenman,Felony assaults have gone up a lot. Shootings ...,Brad Lander,I think it’s the same strategy. It’s like you’...,"If you're talking with Ken Corey, are you worr...",0,0,0
4,Myles Miller,"If you're talking with Ken Corey, are you worr...",Brad Lander,I think the problems with the Caban era of the...,Would you consider keeping Tisch as police com...,0,0,0
...,...,...,...,...,...,...,...,...
451,Akash Mehta,You’re comfortable talking about Zohran Mamdan...,Whitney Tilson,"I guess on a policy basis, I’m reasonably clos...",You ranked Mayor Koch highly. Koch was one of ...,0,0,0
452,Harry Siegel,And this will be our last word to be fair to y...,Whitney Tilson,"I would say, in order, Ukraine, Israel, and Af...",That’s 54 countries.,0,0,0
453,Christina Greer,That’s 54 countries.,Whitney Tilson,"Oh, you’re right. I’ve been to 86 countries, b...",Do you think you can be anti-Zionist and not a...,0,0,0
454,Nicole Gelinas,Do you think you can be anti-Zionist and not a...,Whitney Tilson,"I think it is possible. I think in practice, m...","But is a loud, raucous protest with posters th...",0,0,0


In [15]:
def choose_label(row):
    if row["label_claude"] == row["label_google"]:
        return row["label_claude"]
    else:
        return row["label_openai"]

In [16]:
df["label"] = df.apply(lambda row: choose_label(row), axis=1)

In [17]:
df.to_csv('labeled_data.csv')